### This notebook should create a csv with the Elon Musk Tweets, containg btc releated info, sentimental analisys scores and the btc prices indexed by date

In [1]:
import glob
import os
import pandas as pd
import re
import numpy as np
import string
from sklearn.preprocessing import minmax_scale
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from pprint import pprint

nltk.download('vader_lexicon')

base_path_to_csv = os.path.join(os.getcwd() + '/eltweets/*.csv')
csv_list = glob.glob(base_path_to_csv)

df_list = [pd.read_csv(csv, index_col='id') for csv in csv_list]
df = pd.concat(df_list)
df = df.reset_index().drop_duplicates(subset='id', keep='first').set_index('id')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/prbpedro/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Normalizing tweets to sentimental analisys

df['full_text'] = df['full_text'].astype('unicode')
remove_rt = lambda x: re.sub('RT @\w+: ', ' ', x)
remove_users_ref = lambda x: re.sub("@[A-Za-z0-9]+","",x)
remove_links = lambda x: re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", x)
remove_hashtags_underlines = lambda x: x.replace("#", "").replace("_", " ")

df['full_text'] = df['full_text'].map(remove_rt).map(remove_users_ref)
df['full_text'] = df['full_text'].map(remove_links)
df['full_text'] = df['full_text'].map(remove_hashtags_underlines)

df['full_text'] = df['full_text'].str.lower()
df = df[(
    df['full_text'].str.contains("bitcoin") | 
    df['full_text'].str.contains("btc") | 
    df['full_text'].str.contains("etherum") | 
    df['full_text'].str.contains("eth ") | 
    df['full_text'].str.contains("crypto") | 
    df['full_text'].str.contains("doge"))]
df['full_text'] = df['full_text'].str.replace('&amp;', 'and')
df['full_text'] = df['full_text'].str.replace('&', 'and')
df['full_text'] = df['full_text'].str.replace('💔', 'broke my heart')
df['full_text'] = df['full_text'].str.replace('🤣', 'laughing ')
df['full_text'] = df['full_text'].str.replace('🎶', '')
df['full_text'] = df['full_text'].str.replace("it’s", 'it is')
df['full_text'] = df['full_text'].str.replace("don’t", 'do not')
df['full_text'] = df['full_text'].str.replace("can’t", 'can not')
df['full_text'] = df['full_text'].str.replace("won’t", 'will not')
df['full_text'] = df['full_text'].str.replace("people’s", 'people')
df['full_text'] = df['full_text'].str.replace("people’s", 'people')
df['full_text'] = df['full_text'].str.replace("there’s", 'there is')
 
remove_pontuacao = lambda x:  re.sub(r'[^\w\s]', '', x)
df['full_text'] = df['full_text'].map(remove_pontuacao)

df['full_text'] = df['full_text'].str.replace(r'\\n',' ', regex=True) 

remove_multiplos_espacos = lambda x:  re.sub(' +', ' ', x)
df['full_text'] = df['full_text'].map(remove_multiplos_espacos)

df.dropna(inplace=True)

df['full_text'] = df['full_text'].astype('unicode')

In [3]:
# Normalizing created_at column 
# Creating influence_end_at column containing the date time that indicates the end of the influence of the tweet in the btc price

df['created_at'] = pd.to_datetime(df['created_at'])
df['created_at'] = df['created_at'].dt.normalize()
df['influence_end_at'] = df['created_at']  + pd.DateOffset(days=1)
df['influence_end_at'] = df['influence_end_at'].dt.normalize()

In [4]:
# Creating full_text_score column with the compound value of the sentimental analisys of each tweet

analyzer = SentimentIntensityAnalyzer()
score_full_text = lambda x: analyzer.polarity_scores(x)['compound']

df['full_text_score'] = None
df['full_text_score'] = df['full_text'].map(score_full_text)

In [5]:
# Duplicating rows that contains differents created_at and influence_end_at column values

new_rows = df[df['created_at'] != df['influence_end_at']]
new_rows['created_at'] = new_rows['influence_end_at']
new_rows['id'] = None

new_df = df.append(new_rows)

mask = df['created_at'].duplicated(keep=False)
duplicados = df[mask]

f_df = df[~mask].copy()
f_df['Date'] = f_df['created_at']
f_df['Score'] = f_df['full_text_score']
f_df = f_df[['Date', 'Score']]

In [6]:
# Creating a dictionary with the tweet influence_end_at as key and the sum of the scores of sentimental analisys of tweets of that date and the number of tweets in that day as the value

m = {}
for d in duplicados['created_at'].unique():
    if d not in m.keys():
        m[d] = { 'count': 0, 'full_text_score': 0.0 }
    for i, row in df.loc[df['created_at'] == d].iterrows():
        m[d]['full_text_score'] += row['full_text_score']
        m[d]['count'] += 1

In [7]:
# Creating csv file with columns Date and Score (Arithmetic mean of the twwets sentimental analisys score of that date )

f_df2 = pd.DataFrame([ {'Date': k, 'Score': v['full_text_score'] / v['count'] } for k, v in m.items()])

df = pd.concat([f_df, f_df2], ignore_index=True)
df.set_index('Date', inplace=True, drop=True)
df = df[df['Score'] !=0]
df['Score'] = df['Score'].round(decimals=4)
df.to_csv("btc_em_sentimental_analysis.csv")

In [8]:
# Reading and normalizing csv containing the btc historial prices

btc_df = pd.read_csv("Bitcoin Historical Data - Investing.com.csv", usecols = ['Date','Price'])
btc_df['Date'] = pd.to_datetime(btc_df['Date'])
btc_df['Date'] = btc_df['Date'].dt.normalize()
btc_df = btc_df.reset_index().set_index('Date')
btc_df = btc_df.drop(['index'], axis=1)
btc_df['Price'] = btc_df['Price'].str.replace(',', '')

In [9]:
# Merging the btc historical price data with the sentimental analisys data

df.index = df.index.tz_localize(None)
btc_df.index = btc_df.index.tz_localize(None)
merged = pd.merge(df, btc_df, on = ['Date'], how = 'outer')


In [10]:
# Creating final csv file with Date, Score and Price columns

merged['Score'] = merged['Score'].fillna(0)
merged['Score'] = pd.to_numeric(merged['Score'])
merged['Price'] = pd.to_numeric(merged['Price'])
merged.to_csv('btc_value_em_tweets_sentimental_score.csv')